In [159]:
# 1. We know who is available at first pick, do a simulation for who will be available at later picks
# 2. Optimize all of the selections, but just save the data from the first round selection 
# 3. Repeat the above process 100 times, choosing the player who is selected in round 1 to be the true selection 
# 4. We know who is available at the next pick, do a simulation for later availabilities 
# 4. add a constraint (forcing the first pick to be the person you chose) then optimize all of the selection
# repeat for each player 

using JuMP, Gurobi, CSV, DataFrames, Distributions, Random, StatsBase

bonus_ARI = CSV.read("team_data/ARI_bonus.csv", DataFrame) # bonuses (as proportions!) 
probs_ARI = CSV.read("team_data/ARI_probs.csv", DataFrame) # Used to create simulations
real_ARI = CSV.read("team_data/ARI_real.csv", DataFrame) # binary columns if they are available 
spending_ARI = 3090000+2900000+2150000+990000+650000+800000 # total signing bonuses of Arizona between picks 1 and 136 (before round 5) in the 2024 MLB draft

bonus_ATL = CSV.read("team_data/ATL_bonus.csv", DataFrame)
probs_ATL = CSV.read("team_data/ATL_probs.csv", DataFrame)
real_ATL = CSV.read("team_data/ATL_real.csv", DataFrame)
spending_ATL = (3.56+1.35+.735+.472)*1000000

bonus_BAL = CSV.read("team_data/BAL_bonus.csv", DataFrame)
probs_BAL = CSV.read("team_data/BAL_probs.csv", DataFrame)
real_BAL = CSV.read("team_data/BAL_real.csv", DataFrame)
spending_BAL = (4+2.7+1.17+.850+.5225)*1000000

bonus_BOS = CSV.read("team_data/BOS_bonus.csv", DataFrame)
probs_BOS = CSV.read("team_data/BOS_probs.csv", DataFrame)
real_BOS = CSV.read("team_data/BOS_real.csv", DataFrame)
spending_BOS = (.5+.7+2+5)*1000000

bonus_CHC = CSV.read("team_data/CHC_bonus.csv", DataFrame)
probs_CHC = CSV.read("team_data/CHC_probs.csv", DataFrame)
real_CHC = CSV.read("team_data/CHC_real.csv", DataFrame)
spending_CHC = (1+.62+1.68+5.07)*1000000

bonus_CIN = CSV.read("team_data/CIN_bonus.csv", DataFrame)
probs_CIN = CSV.read("team_data/CIN_probs.csv", DataFrame)
real_CIN = CSV.read("team_data/CIN_real.csv", DataFrame)
spending_CIN = (.625+.863+.9975+3.05+9.25)*1000000

bonus_CLE = CSV.read("team_data/CLE_bonus.csv", DataFrame)
probs_CLE = CSV.read("team_data/CLE_probs.csv", DataFrame)
real_CLE = CSV.read("team_data/CLE_real.csv", DataFrame)
spending_CLE = (.4469+2+2.05+2.57+8.95)*1000000

bonus_COL = CSV.read("team_data/COL_bonus.csv", DataFrame)
probs_COL = CSV.read("team_data/COL_probs.csv", DataFrame)
real_COL = CSV.read("team_data/COL_real.csv", DataFrame)
spending_COL = (.25+1.01+2+2.7+9.25)*1000000

bonus_CHW = CSV.read("team_data/CHW_bonus.csv", DataFrame)
probs_CHW = CSV.read("team_data/CHW_probs.csv", DataFrame)
real_CHW = CSV.read("team_data/CHW_real.csv", DataFrame)
spending_CHW = (.8475+.8+1.4+3+8)*1000000

bonus_DET = CSV.read("team_data/DET_bonus.csv", DataFrame)
probs_DET = CSV.read("team_data/DET_probs.csv", DataFrame)
real_DET = CSV.read("team_data/DET_real.csv", DataFrame)
spending_DET = (.5225+.7+1.8+1.75+5.8)*1000000

bonus_HOU = CSV.read("team_data/HOU_bonus.csv", DataFrame)
probs_HOU = CSV.read("team_data/HOU_probs.csv", DataFrame)
real_HOU = CSV.read("team_data/HOU_real.csv", DataFrame)
spending_HOU = (3.13+1+.4475)*1000000

bonus_KCR = CSV.read("team_data/KCR_bonus.csv", DataFrame)
probs_KCR = CSV.read("team_data/KCR_probs.csv", DataFrame)
real_KCR = CSV.read("team_data/KCR_real.csv", DataFrame)
spending_KCR = (.5975+1.1+2.3+7.5)*1000000

bonus_LAA = CSV.read("team_data/LAA_bonus.csv", DataFrame)
probs_LAA = CSV.read("team_data/LAA_probs.csv", DataFrame)
real_LAA = CSV.read("team_data/LAA_real.csv", DataFrame)
spending_LAA = (.5725+.9486+1.75+1.60+5)*1000000 # considering pick value for pick 81

bonus_LAD = CSV.read("team_data/LAD_bonus.csv", DataFrame)
probs_LAD = CSV.read("team_data/LAD_probs.csv", DataFrame)
real_LAD = CSV.read("team_data/LAD_real.csv", DataFrame)
spending_LAD = (.5538+1.75+3.3)*1000000

bonus_MIA = CSV.read("team_data/MIA_bonus.csv", DataFrame)
probs_MIA = CSV.read("team_data/MIA_probs.csv", DataFrame)
real_MIA = CSV.read("team_data/MIA_real.csv", DataFrame)
spending_MIA = (.55+.8+.9+2.8+3.4)*1000000

bonus_MIL = CSV.read("team_data/MIL_bonus.csv", DataFrame)
probs_MIL = CSV.read("team_data/MIL_probs.csv", DataFrame)
real_MIL = CSV.read("team_data/MIL_real.csv", DataFrame)
spending_MIL = (.5+.025+1.23+2.5+2.1+3.44)*1000000 # considering pick value for pick 67

bonus_MIN = CSV.read("team_data/MIN_bonus.csv", DataFrame)
probs_MIN = CSV.read("team_data/MIN_probs.csv", DataFrame)
real_MIN = CSV.read("team_data/MIN_real.csv", DataFrame)
spending_MIN = (.425+.5975+2+1.45+2.4+3.93)*1000000

bonus_NYM = CSV.read("team_data/NYM_bonus.csv", DataFrame)
probs_NYM = CSV.read("team_data/NYM_probs.csv", DataFrame)
real_NYM = CSV.read("team_data/NYM_real.csv", DataFrame)
spending_NYM = (.6975+.7975+2.03+4)*1000000

bonus_NYY = CSV.read("team_data/NYY_bonus.csv", DataFrame)
probs_NYY = CSV.read("team_data/NYY_probs.csv", DataFrame)
real_NYY = CSV.read("team_data/NYY_real.csv", DataFrame)
spending_NYY = (.637+.836+2.3+2.75)*1000000

bonus_OAK = CSV.read("team_data/OAK_bonus.csv", DataFrame)
probs_OAK = CSV.read("team_data/OAK_probs.csv", DataFrame)
real_OAK = CSV.read("team_data/OAK_real.csv", DataFrame)
spending_OAK = (1+1.04+2+3+7)*1000000

bonus_PHI = CSV.read("team_data/PHI_bonus.csv", DataFrame)
probs_PHI = CSV.read("team_data/PHI_probs.csv", DataFrame)
real_PHI = CSV.read("team_data/PHI_real.csv", DataFrame)
spending_PHI = (.545+.6725+2.5+2.5)*1000000

bonus_PIT = CSV.read("team_data/PIT_bonus.csv", DataFrame)
probs_PIT = CSV.read("team_data/PIT_probs.csv", DataFrame)
real_PIT = CSV.read("team_data/PIT_real.csv", DataFrame)
spending_PIT = (.6497+.85+2.5+2.51+6.53)*1000000

bonus_SDP = CSV.read("team_data/SDP_bonus.csv", DataFrame)
probs_SDP = CSV.read("team_data/SDP_probs.csv", DataFrame)
real_SDP = CSV.read("team_data/SDP_real.csv", DataFrame)
spending_SDP = (.08+.525+.6+.8523+2.5+3.44)*1000000

bonus_SEA = CSV.read("team_data/SEA_bonus.csv", DataFrame)
probs_SEA = CSV.read("team_data/SEA_probs.csv", DataFrame)
real_SEA = CSV.read("team_data/SEA_real.csv", DataFrame)
spending_SEA = (.594+.05+3+4.88)*1000000

bonus_SFG = CSV.read("team_data/SFG_bonus.csv", DataFrame)
probs_SFG = CSV.read("team_data/SFG_probs.csv", DataFrame)
real_SFG = CSV.read("team_data/SFG_real.csv", DataFrame)
spending_SFG = (2+4.74)*1000000

bonus_STL = CSV.read("team_data/STL_bonus.csv", DataFrame)
probs_STL = CSV.read("team_data/STL_probs.csv", DataFrame)
real_STL = CSV.read("team_data/STL_real.csv", DataFrame)
spending_STL = (.6693+.8+6.9)*1000000

bonus_TBR = CSV.read("team_data/TBR_bonus.csv", DataFrame)
probs_TBR = CSV.read("team_data/TBR_probs.csv", DataFrame)
real_TBR = CSV.read("team_data/TBR_real.csv", DataFrame)
spending_TBR = (.397+.774+1.26+1.52+4.37)*1000000 # using pick value for pick 66

bonus_TEX = CSV.read("team_data/TEX_bonus.csv", DataFrame)
probs_TEX = CSV.read("team_data/TEX_probs.csv", DataFrame)
real_TEX = CSV.read("team_data/TEX_real.csv", DataFrame)
spending_TEX = (.515+.7+1.29+3)*1000000

bonus_TOR = CSV.read("team_data/TOR_bonus.csv", DataFrame)
probs_TOR = CSV.read("team_data/TOR_probs.csv", DataFrame)
real_TOR = CSV.read("team_data/TOR_real.csv", DataFrame)
spending_TOR = (.4675+.5697+1.25+1.12+4.18)*1000000

bonus_WSN = CSV.read("team_data/WSN_bonus.csv", DataFrame)
probs_WSN = CSV.read("team_data/WSN_probs.csv", DataFrame)
real_WSN = CSV.read("team_data/WSN_real.csv", DataFrame)
spending_WSN = (.625+.9803+3.8+2.33+5.15)*1000000

1.28853e7

In [160]:
# 1 do one simulation of who is avaible at later picks:

function simulation(pick_number, probs_df, real_df)

    sim_df = DataFrame() #create a new blank dataframe 
    counter = 0 
    prev_col = ""
    for col in names(probs_df)[2:end]  # Skip the name column
        counter = counter +1
        if counter <= pick_number # because we only simulate future availabilites
            sim_df[!,col] = real_df[!,counter+1] # we know the real availability of current pick, it is one column greater than pick number (b/c name)
        else
            new_col =  Vector{Int}(undef, nrow(probs_df))  # Predefine new col
            
            for row in 1:nrow(probs_df)
                if sim_df[row,prev_col]!=0 # if a player is potentially available (ie not zero for the round before) 
                    new_col[row] = rand(Bernoulli(probs_df[row, col]))
                else
                    new_col[row]=0
                
                end
            end
            sim_df[!,col]=new_col #now, we put the new data in 
        end
        prev_col = col 
    end
    
    sim_df.name .= ""
    sim_df[!,:name] = real_df[!,:name]

    return sim_df

end

simulation (generic function with 1 method)

In [161]:
# 2. Optimize all of the selections, but just save the data from the first round selection 

function do_optimization(the_round, probs_df, real_df, bonus_df, risk_averse, hs_averse, spending, list_of_selected_players)

    player_data = CSV.read("data_fg - 2024.csv", DataFrame) 
    final_sim = simulation(the_round,probs_df, real_df)
    df = innerjoin(final_sim, player_data, on=:name)
    df = sort(df, :name)
    bonus_df = sort(bonus_df, :name)
    if nrow(df) != nrow(bonus_df)
        print("DFs DO NOT ALIGN!")
    end

    # final_sim and player_data on the shared column name

    num_players = nrow(df)
    num_rounds = ncol(final_sim)-1 # so we don't count the name column 
    lambda_1 = risk_averse #Risk averse-ness
    lambda_2 = hs_averse #High School averse-ness
    budget = (spending) / 9250000  # as proportion of most expensive draft pick in draft, which was 9,250,000 in 2024 
    # we also took into account the real budget for the dbacks over that relevant draft picks 

    # Extract relevant columns
    FV = df.fv
    Risk = df.risk
    HighSchool = df.hs
    # DON'T FORGET TO DO THE BONUS COLUMN ! 

    # Define the model
    model_opt = Model(Gurobi.Optimizer)
    set_silent(model_opt)
    #model_opt = Model(() -> Gurobi.Optimizer(OutputFlag=0)) # to avoid the long printed outptu


    @variable(model_opt, x[1:num_players, 1:num_rounds], Bin) # decision variable is if you take player p in round r 

    # Constraints
    @constraint(model_opt, [i in 1:num_players], sum(x[i, r] for r in 1:num_rounds) <= 1) # each player is chosen in at most 1 round 
    @constraint(model_opt, [r in 1:num_rounds], sum(x[i, r] for i in 1:num_players) == 1) # you take exactly one player per round  

    # if a player is unavailable in that round, we don't take them 
    for i in 1:num_players, r in 1:num_rounds
        if df[i,r] == 0
            @constraint(model_opt, x[i, r] == 0)
        end
    end

    # Budget constraint 
    @expression(model_opt, total_pay, sum(bonus_df[i, r+1] * x[i, r] for i in 1:num_players, r in 1:num_rounds)) # calculate total_pay
    @constraint(model_opt, total_pay <= budget) # total pay less than 


    # position limit 
    position_list = ["C", "1B", "IF", "CF", "OF", "RHP", "LHP"]
    for pos in position_list
        @constraint(model_opt,
            sum(x[i, r] for i in 1:num_players, r in 1:num_rounds if df.position[i] == pos) <= 2  # we only want to take at most 2 of the same position 
        )
    end

    # if we already have selected a player, we must force the selection 
    for s in 1:length(list_of_selected_players)
        the_selected_player = list_of_selected_players[s]
        for i in 1:num_players
            if df.name[i] == the_selected_player
                @constraint(model_opt, x[i,s] == 1)
            end
        end
    end
    

    # Objective function
    @objective(model_opt, Max,
        sum(x[i, r] * (FV[i] - lambda_1 * Risk[i] - lambda_2 * HighSchool[i]) for i in 1:num_players, r in 1:num_rounds)
    )

    optimize!(model_opt)
    
    # Output the results
    the_player = ""
    for i in 1:num_players
        for r in 1:num_rounds
            if value(x[i, r]) >= 0.1
                if r == the_round 
                    #println("Player: ", df.name[i], " in round: ", r)
                    the_player = df.name[i]
                end
            end
        end
    end

    return the_player 

end

do_optimization (generic function with 2 methods)

In [162]:
function run_the_model(probs_df, real_df,bonus_df, penalty_risk, penalty_hs,relevant_budget,iterations)


    selection_list = []
    for pick in 1:(ncol(probs_df)-1)
        player_list = []
        for i in 1:iterations
            new_player = do_optimization(pick, probs_df, real_df, bonus_df, penalty_risk, penalty_hs, relevant_budget, selection_list)
             push!(player_list, new_player)
        end
        # count how often each player picked
        counts = countmap(player_list)
        sorted_counts = sort(collect(countmap(player_list)), by = x -> x[2], rev = true)
        
        # Print results
        for (value, count) in sorted_counts
            println("$(value): $(count)")
        end
        selected_player = sorted_counts[1][1]
        push!(selection_list,selected_player)
    end
    
    return selection_list
end


run_the_model (generic function with 2 methods)

In [173]:
# for current playoff and big market bubble teams (as of draft day, july 14 2024) we make high school penalty 0.25

output_ARI = run_the_model(probs_ARI, real_ARI, bonus_ARI, 0.5, 0.0, spending_ARI, 100)
output_ATL = run_the_model(probs_ATL, real_ATL, bonus_ATL, 0.5, 10, spending_ATL, 100)
output_BAL = run_the_model(probs_BAL, real_BAL, bonus_BAL, 0.5, 10, spending_BAL, 100)
output_BOS = run_the_model(probs_BOS, real_BOS, bonus_BOS, 0.5, 10, spending_BOS, 100)
output_CHC = run_the_model(probs_CHC, real_CHC, bonus_CHC, 0.5, 0.0, spending_CHC, 100)
output_CIN = run_the_model(probs_CIN, real_CIN, bonus_CIN, 0.5, 0.0, spending_CIN, 100)
output_CLE = run_the_model(probs_CLE, real_CLE, bonus_CLE, 0.5, 10, spending_CLE, 100)
output_COL = run_the_model(probs_COL, real_COL, bonus_COL, 0.5, 0.0, spending_COL, 100)
output_CHW = run_the_model(probs_CHW, real_CHW, bonus_CHW, 0.5, 0.0, spending_CHW, 100)
output_DET = run_the_model(probs_DET, real_DET, bonus_DET, 0.5, 0.0, spending_DET, 100)
output_HOU = run_the_model(probs_HOU, real_HOU, bonus_HOU, 0.5, 10, spending_HOU, 100)
output_KCR = run_the_model(probs_KCR, real_KCR, bonus_KCR, 0.5, 0.0, spending_KCR, 100)
output_LAA = run_the_model(probs_LAA, real_LAA, bonus_LAA, 0.5, 0.0, spending_LAA, 100)
output_LAD = run_the_model(probs_LAD, real_LAD, bonus_LAD, 0.5, 10, spending_LAD, 100)
output_MIA = run_the_model(probs_MIA, real_MIA, bonus_MIA, 0.5, 0.0, spending_MIA, 100)
output_MIL = run_the_model(probs_MIL, real_MIL, bonus_MIL, 0.5, 10, spending_MIL, 100)
output_MIN = run_the_model(probs_MIN, real_MIN, bonus_MIN, 0.5, 10, spending_MIN, 100)
output_NYM = run_the_model(probs_NYM, real_NYM, bonus_NYM, 0.5, 10, spending_NYM, 100)
output_NYY = run_the_model(probs_NYY, real_NYY, bonus_NYY, 0.5, 10, spending_NYY, 100)
output_OAK = run_the_model(probs_OAK, real_OAK, bonus_OAK, 0.5, 0.0, spending_OAK, 100)
output_PHI = run_the_model(probs_PHI, real_PHI, bonus_PHI, 0.5, 10, spending_PHI, 100)
output_PIT = run_the_model(probs_PIT, real_PIT, bonus_PIT, 0.5, 0.0, spending_PIT, 100)
output_SDP = run_the_model(probs_SDP, real_SDP, bonus_SDP, 0.5, 10, spending_SDP, 100)
output_SEA = run_the_model(probs_SEA, real_SEA, bonus_SEA, 0.5, 10, spending_SEA, 100)
output_SFG = run_the_model(probs_SFG, real_SFG, bonus_SFG, 0.5, 0.0, spending_SFG, 100)
output_STL = run_the_model(probs_STL, real_STL, bonus_STL, 0.5, 10, spending_STL, 100)
output_TBR = run_the_model(probs_TBR, real_TBR, bonus_TBR, 0.5, 0.0, spending_TBR, 100)
output_TEX = run_the_model(probs_TEX, real_TEX, bonus_TEX, 0.5, 0.0, spending_TEX, 100)
output_TOR = run_the_model(probs_TOR, real_TOR, bonus_TOR, 0.5, 0.0, spending_TOR, 100)
output_WSN = run_the_model(probs_WSN, real_WSN, bonus_WSN, 0.5, 0.0, spending_WSN, 100)


# Combine all outputs into a single vector
all_outputs = vcat(
    output_ARI, output_ATL, output_BAL, output_BOS, output_CHC, output_CIN, output_CLE, output_COL, output_CHW,
    output_DET, output_HOU, output_KCR,  output_LAA, output_LAD, output_MIA, output_MIL, output_MIN, output_NYM,
    output_NYY, output_OAK, output_PHI, output_PIT, output_SDP, output_SEA, output_SFG, output_STL, output_TBR,
    output_TEX, output_TOR, output_WSN
)

# Sort descending by count
sorted_counts_all = sort(collect(countmap(all_outputs)), by = x -> x[2], rev = true)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-21

Excessive output truncated after 533074 bytes.

20-element Vector{Pair{Any, Int64}}:
     String31("Kavares Tears") => 29
     String31("David Hagaman") => 29
      String31("Brody Brecht") => 19
     String31("Wyatt Sanford") => 7
  String31("Carson DeMartini") => 6
    String31("Brandon Clarke") => 6
 String31("Braden Montgomery") => 5
        String31("Dasan Hill") => 4
      String31("Braden Davis") => 4
     String31("Ryan Stafford") => 4
    String31("Jac Caglianone") => 3
    String31("Caleb Lomavita") => 3
       String31("Luke Hayden") => 3
    String31("Charlie Condon") => 3
       String31("Tyson Lewis") => 3
         String31("Cam Smith") => 3
     String31("Dakota Jordan") => 2
        String31("Joey Oakie") => 1
  String31("Bryce Cunningham") => 1
   String31("Vance Honeycutt") => 1

In [174]:
highest_bonus = []

bonuses_ARI = Float64[]  

for i in 1:length(output_ARI)
    player_name = output_ARI[i]
    row_idx = findfirst(x -> x == player_name, bonus_ARI[!, 1])  # match name in column 1
    push!(bonuses_ARI, bonus_ARI[row_idx, i + 1])  # i+1 because column 1 is names
end

df_with_bonuses_ARI = DataFrame(Name = output_ARI, Bonus = bonuses)
push!(highest_bonus,argmax(df_with_bonuses_ARI.Bonus))


bonuses_ATL = Float64[]
for i in 1:length(output_ATL)
    player_name = output_ATL[i]
    row_idx = findfirst(x -> x == player_name, bonus_ATL[!, 1])
    push!(bonuses_ATL, bonus_ATL[row_idx, i + 1])
end
df_with_bonuses_ATL = DataFrame(Name = output_ATL, Bonus = bonuses_ATL)
push!(highest_bonus, argmax(df_with_bonuses_ATL.Bonus))

bonuses_BAL = Float64[]
for i in 1:length(output_BAL)
    player_name = output_BAL[i]
    row_idx = findfirst(x -> x == player_name, bonus_BAL[!, 1])
    push!(bonuses_BAL, bonus_BAL[row_idx, i + 1])
end
df_with_bonuses_BAL = DataFrame(Name = output_BAL, Bonus = bonuses_BAL)
push!(highest_bonus, argmax(df_with_bonuses_BAL.Bonus))

bonuses_BOS = Float64[]
for i in 1:length(output_BOS)
    player_name = output_BOS[i]
    row_idx = findfirst(x -> x == player_name, bonus_BOS[!, 1])
    push!(bonuses_BOS, bonus_BOS[row_idx, i + 1])
end
df_with_bonuses_BOS = DataFrame(Name = output_BOS, Bonus = bonuses_BOS)
push!(highest_bonus, argmax(df_with_bonuses_BOS.Bonus))

bonuses_CHC = Float64[]
for i in 1:length(output_CHC)
    player_name = output_CHC[i]
    row_idx = findfirst(x -> x == player_name, bonus_CHC[!, 1])
    push!(bonuses_CHC, bonus_CHC[row_idx, i + 1])
end
df_with_bonuses_CHC = DataFrame(Name = output_CHC, Bonus = bonuses_CHC)
push!(highest_bonus, argmax(df_with_bonuses_CHC.Bonus))

bonuses_CIN = Float64[]
for i in 1:length(output_CIN)
    player_name = output_CIN[i]
    row_idx = findfirst(x -> x == player_name, bonus_CIN[!, 1])
    push!(bonuses_CIN, bonus_CIN[row_idx, i + 1])
end
df_with_bonuses_CIN = DataFrame(Name = output_CIN, Bonus = bonuses_CIN)
push!(highest_bonus, argmax(df_with_bonuses_CIN.Bonus))

bonuses_CLE = Float64[]
for i in 1:length(output_CLE)
    player_name = output_CLE[i]
    row_idx = findfirst(x -> x == player_name, bonus_CLE[!, 1])
    push!(bonuses_CLE, bonus_CLE[row_idx, i + 1])
end
df_with_bonuses_CLE = DataFrame(Name = output_CLE, Bonus = bonuses_CLE)
push!(highest_bonus, argmax(df_with_bonuses_CLE.Bonus))

bonuses_COL = Float64[]
for i in 1:length(output_COL)
    player_name = output_COL[i]
    row_idx = findfirst(x -> x == player_name, bonus_COL[!, 1])
    push!(bonuses_COL, bonus_COL[row_idx, i + 1])
end
df_with_bonuses_COL = DataFrame(Name = output_COL, Bonus = bonuses_COL)
push!(highest_bonus, argmax(df_with_bonuses_COL.Bonus))

bonuses_CHW = Float64[]
for i in 1:length(output_CHW)
    player_name = output_CHW[i]
    row_idx = findfirst(x -> x == player_name, bonus_CHW[!, 1])
    push!(bonuses_CHW, bonus_CHW[row_idx, i + 1])
end
df_with_bonuses_CHW = DataFrame(Name = output_CHW, Bonus = bonuses_CHW)
push!(highest_bonus, argmax(df_with_bonuses_CHW.Bonus))

bonuses_DET = Float64[]
for i in 1:length(output_DET)
    player_name = output_DET[i]
    row_idx = findfirst(x -> x == player_name, bonus_DET[!, 1])
    push!(bonuses_DET, bonus_DET[row_idx, i + 1])
end
df_with_bonuses_DET = DataFrame(Name = output_DET, Bonus = bonuses_DET)
push!(highest_bonus, argmax(df_with_bonuses_DET.Bonus))

bonuses_HOU = Float64[]
for i in 1:length(output_HOU)
    player_name = output_HOU[i]
    row_idx = findfirst(x -> x == player_name, bonus_HOU[!, 1])
    push!(bonuses_HOU, bonus_HOU[row_idx, i + 1])
end
df_with_bonuses_HOU = DataFrame(Name = output_HOU, Bonus = bonuses_HOU)
push!(highest_bonus, argmax(df_with_bonuses_HOU.Bonus))

bonuses_KCR = Float64[]
for i in 1:length(output_KCR)
    player_name = output_KCR[i]
    row_idx = findfirst(x -> x == player_name, bonus_KCR[!, 1])
    push!(bonuses_KCR, bonus_KCR[row_idx, i + 1])
end
df_with_bonuses_KCR = DataFrame(Name = output_KCR, Bonus = bonuses_KCR)
push!(highest_bonus, argmax(df_with_bonuses_KCR.Bonus))

bonuses_LAA = Float64[]
for i in 1:length(output_LAA)
    player_name = output_LAA[i]
    row_idx = findfirst(x -> x == player_name, bonus_LAA[!, 1])
    push!(bonuses_LAA, bonus_LAA[row_idx, i + 1])
end
df_with_bonuses_LAA = DataFrame(Name = output_LAA, Bonus = bonuses_LAA)
push!(highest_bonus, argmax(df_with_bonuses_LAA.Bonus))

bonuses_LAD = Float64[]
for i in 1:length(output_LAD)
    player_name = output_LAD[i]
    row_idx = findfirst(x -> x == player_name, bonus_LAD[!, 1])
    push!(bonuses_LAD, bonus_LAD[row_idx, i + 1])
end
df_with_bonuses_LAD = DataFrame(Name = output_LAD, Bonus = bonuses_LAD)
push!(highest_bonus, argmax(df_with_bonuses_LAD.Bonus))

bonuses_MIA = Float64[]
for i in 1:length(output_MIA)
    player_name = output_MIA[i]
    row_idx = findfirst(x -> x == player_name, bonus_MIA[!, 1])
    push!(bonuses_MIA, bonus_MIA[row_idx, i + 1])
end
df_with_bonuses_MIA = DataFrame(Name = output_MIA, Bonus = bonuses_MIA)
push!(highest_bonus, argmax(df_with_bonuses_MIA.Bonus))

bonuses_MIL = Float64[]
for i in 1:length(output_MIL)
    player_name = output_MIL[i]
    row_idx = findfirst(x -> x == player_name, bonus_MIL[!, 1])
    push!(bonuses_MIL, bonus_MIL[row_idx, i + 1])
end
df_with_bonuses_MIL = DataFrame(Name = output_MIL, Bonus = bonuses_MIL)
push!(highest_bonus, argmax(df_with_bonuses_MIL.Bonus))

bonuses_MIN = Float64[]
for i in 1:length(output_MIN)
    player_name = output_MIN[i]
    row_idx = findfirst(x -> x == player_name, bonus_MIN[!, 1])
    push!(bonuses_MIN, bonus_MIN[row_idx, i + 1])
end
df_with_bonuses_MIN = DataFrame(Name = output_MIN, Bonus = bonuses_MIN)
push!(highest_bonus, argmax(df_with_bonuses_MIN.Bonus))

bonuses_NYM = Float64[]
for i in 1:length(output_NYM)
    player_name = output_NYM[i]
    row_idx = findfirst(x -> x == player_name, bonus_NYM[!, 1])
    push!(bonuses_NYM, bonus_NYM[row_idx, i + 1])
end
df_with_bonuses_NYM = DataFrame(Name = output_NYM, Bonus = bonuses_NYM)
push!(highest_bonus, argmax(df_with_bonuses_NYM.Bonus))

bonuses_NYY = Float64[]
for i in 1:length(output_NYY)
    player_name = output_NYY[i]
    row_idx = findfirst(x -> x == player_name, bonus_NYY[!, 1])
    push!(bonuses_NYY, bonus_NYY[row_idx, i + 1])
end
df_with_bonuses_NYY = DataFrame(Name = output_NYY, Bonus = bonuses_NYY)
push!(highest_bonus, argmax(df_with_bonuses_NYY.Bonus))

bonuses_OAK = Float64[]
for i in 1:length(output_OAK)
    player_name = output_OAK[i]
    row_idx = findfirst(x -> x == player_name, bonus_OAK[!, 1])
    push!(bonuses_OAK, bonus_OAK[row_idx, i + 1])
end
df_with_bonuses_OAK = DataFrame(Name = output_OAK, Bonus = bonuses_OAK)
push!(highest_bonus, argmax(df_with_bonuses_OAK.Bonus))

bonuses_PHI = Float64[]
for i in 1:length(output_PHI)
    player_name = output_PHI[i]
    row_idx = findfirst(x -> x == player_name, bonus_PHI[!, 1])
    push!(bonuses_PHI, bonus_PHI[row_idx, i + 1])
end
df_with_bonuses_PHI = DataFrame(Name = output_PHI, Bonus = bonuses_PHI)
push!(highest_bonus, argmax(df_with_bonuses_PHI.Bonus))

bonuses_PIT = Float64[]
for i in 1:length(output_PIT)
    player_name = output_PIT[i]
    row_idx = findfirst(x -> x == player_name, bonus_PIT[!, 1])
    push!(bonuses_PIT, bonus_PIT[row_idx, i + 1])
end
df_with_bonuses_PIT = DataFrame(Name = output_PIT, Bonus = bonuses_PIT)
push!(highest_bonus, argmax(df_with_bonuses_PIT.Bonus))

bonuses_SDP = Float64[]
for i in 1:length(output_SDP)
    player_name = output_SDP[i]
    row_idx = findfirst(x -> x == player_name, bonus_SDP[!, 1])
    push!(bonuses_SDP, bonus_SDP[row_idx, i + 1])
end
df_with_bonuses_SDP = DataFrame(Name = output_SDP, Bonus = bonuses_SDP)
push!(highest_bonus, argmax(df_with_bonuses_SDP.Bonus))

bonuses_SEA = Float64[]
for i in 1:length(output_SEA)
    player_name = output_SEA[i]
    row_idx = findfirst(x -> x == player_name, bonus_SEA[!, 1])
    push!(bonuses_SEA, bonus_SEA[row_idx, i + 1])
end
df_with_bonuses_SEA = DataFrame(Name = output_SEA, Bonus = bonuses_SEA)
push!(highest_bonus, argmax(df_with_bonuses_SEA.Bonus))

bonuses_SFG = Float64[]
for i in 1:length(output_SFG)
    player_name = output_SFG[i]
    row_idx = findfirst(x -> x == player_name, bonus_SFG[!, 1])
    push!(bonuses_SFG, bonus_SFG[row_idx, i + 1])
end
df_with_bonuses_SFG = DataFrame(Name = output_SFG, Bonus = bonuses_SFG)
push!(highest_bonus, argmax(df_with_bonuses_SFG.Bonus))

bonuses_STL = Float64[]
for i in 1:length(output_STL)
    player_name = output_STL[i]
    row_idx = findfirst(x -> x == player_name, bonus_STL[!, 1])
    push!(bonuses_STL, bonus_STL[row_idx, i + 1])
end
df_with_bonuses_STL = DataFrame(Name = output_STL, Bonus = bonuses_STL)
push!(highest_bonus, argmax(df_with_bonuses_STL.Bonus))

bonuses_TBR = Float64[]
for i in 1:length(output_TBR)
    player_name = output_TBR[i]
    row_idx = findfirst(x -> x == player_name, bonus_TBR[!, 1])
    push!(bonuses_TBR, bonus_TBR[row_idx, i + 1])
end
df_with_bonuses_TBR = DataFrame(Name = output_TBR, Bonus = bonuses_TBR)
push!(highest_bonus, argmax(df_with_bonuses_TBR.Bonus))

bonuses_TEX = Float64[]
for i in 1:length(output_TEX)
    player_name = output_TEX[i]
    row_idx = findfirst(x -> x == player_name, bonus_TEX[!, 1])
    push!(bonuses_TEX, bonus_TEX[row_idx, i + 1])
end
df_with_bonuses_TEX = DataFrame(Name = output_TEX, Bonus = bonuses_TEX)
push!(highest_bonus, argmax(df_with_bonuses_TEX.Bonus))

bonuses_TOR = Float64[]
for i in 1:length(output_TOR)
    player_name = output_TOR[i]
    row_idx = findfirst(x -> x == player_name, bonus_TOR[!, 1])
    push!(bonuses_TOR, bonus_TOR[row_idx, i + 1])
end
df_with_bonuses_TOR = DataFrame(Name = output_TOR, Bonus = bonuses_TOR)
push!(highest_bonus, argmax(df_with_bonuses_TOR.Bonus))

bonuses_WSN = Float64[]
for i in 1:length(output_WSN)
    player_name = output_WSN[i]
    row_idx = findfirst(x -> x == player_name, bonus_WSN[!, 1])
    push!(bonuses_WSN, bonus_WSN[row_idx, i + 1])
end
df_with_bonuses_WSN = DataFrame(Name = output_WSN, Bonus = bonuses_WSN)
push!(highest_bonus, argmax(df_with_bonuses_WSN.Bonus))

sorted_counts_bonuses = sort(collect(countmap(highest_bonus)), by = x -> -x[2])  # sort by count descending

2-element Vector{Pair{Any, Int64}}:
 1 => 29
 3 => 1

In [179]:
# writing outputs: 

CSV.write("optimized_data/output_ARI.csv", DataFrame(name = df_with_bonuses_ARI))
CSV.write("optimized_data/output_ATL.csv", DataFrame(name = df_with_bonuses_ATL))
CSV.write("optimized_data/output_BAL.csv", DataFrame(name = df_with_bonuses_BAL))
CSV.write("optimized_data/output_BOS.csv", DataFrame(name = df_with_bonuses_BOS))
CSV.write("optimized_data/output_CHC.csv", DataFrame(name = df_with_bonuses_CHC))
CSV.write("optimized_data/output_CIN.csv", DataFrame(name = df_with_bonuses_CIN))
CSV.write("optimized_data/output_CLE.csv", DataFrame(name = df_with_bonuses_CLE))
CSV.write("optimized_data/output_COL.csv", DataFrame(name = df_with_bonuses_COL))
CSV.write("optimized_data/output_CHW.csv", DataFrame(name = df_with_bonuses_CHW))
CSV.write("optimized_data/output_DET.csv", DataFrame(name = df_with_bonuses_DET))
CSV.write("optimized_data/output_HOU.csv", DataFrame(name = df_with_bonuses_HOU))
CSV.write("optimized_data/output_KCR.csv",  DataFrame(name = df_with_bonuses_KCR))
CSV.write("optimized_data/output_LAA.csv", DataFrame(name = df_with_bonuses_LAA))
CSV.write("optimized_data/output_LAD.csv", DataFrame(name =df_with_bonuses_LAD))
CSV.write("optimized_data/output_MIA.csv", DataFrame(name = df_with_bonuses_MIA))
CSV.write("optimized_data/output_MIL.csv", DataFrame(name = df_with_bonuses_MIL))
CSV.write("optimized_data/output_MIN.csv", DataFrame(name = df_with_bonuses_MIN))
CSV.write("optimized_data/output_NYM.csv", DataFrame(name = df_with_bonuses_NYM))
CSV.write("optimized_data/output_NYY.csv", DataFrame(name = df_with_bonuses_NYY))
CSV.write("optimized_data/output_OAK.csv", DataFrame(name = df_with_bonuses_OAK))
CSV.write("optimized_data/output_PHI.csv", DataFrame(name = df_with_bonuses_PHI))
CSV.write("optimized_data/output_PIT.csv", DataFrame(name = df_with_bonuses_PIT))
CSV.write("optimized_data/output_SDP.csv", DataFrame(name = df_with_bonuses_SDP))
CSV.write("optimized_data/output_SEA.csv", DataFrame(name = df_with_bonuses_SEA))
CSV.write("optimized_data/output_SFG.csv", DataFrame(name = df_with_bonuses_SFG))
CSV.write("optimized_data/output_STL.csv", DataFrame(name =df_with_bonuses_STL))
CSV.write("optimized_data/output_TBR.csv", DataFrame(name = df_with_bonuses_TBR))
CSV.write("optimized_data/output_TEX.csv", DataFrame(name = df_with_bonuses_TEX))
CSV.write("optimized_data/output_TOR.csv", DataFrame(name = df_with_bonuses_TOR))
CSV.write("optimized_data/output_WSN.csv", DataFrame(name = df_with_bonuses_WSN))

CSV.write("optimized_data/most_popular.csv", DataFrame(name = sorted_counts_all))
CSV.write("optimized_data/highest_bonuses.csv", DataFrame(name = highest_bonus))
CSV.write("optimized_data/highest_bonus_counts.csv", DataFrame(name = sorted_counts_bonuses))

"optimized_data/highest_bonus_counts.csv"

In [ ]:
# team outputs: selections for every team, and signing bonuses for each selection 
# comparisons to real life selections (need a df with team, player, and bonuses) (data_ba results.csv)

# overall outputs: most popular selection, and percent of teams who gave higher bonuses to later picks 
# what team chose the popular selections in reality 

# next steps: verify the accuracy of signing bonus data
# think about some ways to incease diversity (hs penalties for playoff teams) (MAYBE unique position limits?)
# create csv outputs (one for each team, and one for sorted_counts_all 
# verify that the results are somewhat reasonable for each team 
# then, work on streamlit dashboard 

In [175]:
sorted_counts_all

20-element Vector{Pair{Any, Int64}}:
     String31("Kavares Tears") => 29
     String31("David Hagaman") => 29
      String31("Brody Brecht") => 19
     String31("Wyatt Sanford") => 7
  String31("Carson DeMartini") => 6
    String31("Brandon Clarke") => 6
 String31("Braden Montgomery") => 5
        String31("Dasan Hill") => 4
      String31("Braden Davis") => 4
     String31("Ryan Stafford") => 4
    String31("Jac Caglianone") => 3
    String31("Caleb Lomavita") => 3
       String31("Luke Hayden") => 3
    String31("Charlie Condon") => 3
       String31("Tyson Lewis") => 3
         String31("Cam Smith") => 3
     String31("Dakota Jordan") => 2
        String31("Joey Oakie") => 1
  String31("Bryce Cunningham") => 1
   String31("Vance Honeycutt") => 1

In [180]:
CSV.write("optimized_data/output_ARI.csv", DataFrame(name = df_with_bonuses_ARI))
df_with_bonuses_ARI

Row,Name,Bonus
,Any,Float64
1,Brody Brecht,0.274684
2,Caleb Lomavita,0.24681
3,Wyatt Sanford,0.304718
4,David Hagaman,0.139983
5,Kavares Tears,0.0714915
6,Brandon Clarke,0.0465997


In [177]:
highest_bonus

30-element Vector{Any}:
 3
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [178]:
sorted_counts_bonuses

2-element Vector{Pair{Any, Int64}}:
 1 => 29
 3 => 1